# gpd

This notebook includes [GPD](https://github.com/interseismic/generalized-phase-detection) definition, testing and training code.

## Model Definition

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
def gpd(n_samples = 400, n_channels = 3, n_classes = 3, flatten = True):
    
    inputs = layers.Input(shape=(n_samples, n_channels))
    
    # CNN
    n_filters = [32, 64, 128, 256]
    s_kernels = [21, 15, 11, 9]

    x = inputs
    for n_filter, s_kernel in zip(n_filters, s_kernels):

      x = layers.Conv1D(filters = n_filter, kernel_size = s_kernel, padding = 'same', activation = None)(x)
      x = layers.MaxPooling1D()(x)
      x = layers.Activation('relu')(x)
      x = layers.BatchNormalization()(x)
    
    # Concatenate to a single vector
    if flatten:
        x = layers.Flatten()(x)
    else:
        x = layers.GlobalAveragePooling1D()(x)
    
    # FCNN
    for _ in range(2):
      x = layers.Dense(200, activation = None)(x)
      x = layers.Activation('relu')(x)
      x = layers.BatchNormalization()(x)
    
    outputs = layers.Dense(n_classes, activation = 'softmax')(x)
    
    return keras.Model(inputs, outputs)

## Load Model

In [5]:
model = gpd()

model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()]
)

model.load_weights('C:/data/weights/w_gpd_scsn_2000_2017.h5')

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 3)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 400, 32)           2048      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 200, 32)           0         
_________________________________________________________________
activation (Activation)      (None, 200, 32)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 32)           128       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 64)           30784     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 64)           0     

## Test Data

In [6]:
# Modifying sys.path to be able to load project packages
import sys
sys.path.append('../')

In [7]:
from h5_generator import train_test_split as tts

In [31]:
_, X_test = tts('C:/data/datasets/sh_bh_datasets/2014_2021_global_norm_sh_bh.h5', 
                batch_size = 100, 
                shuffle = False, 
                test_size = 1.)

In [32]:
results = model.evaluate(X_test)

string = ''
for metric, value in zip(model.metrics_names, results):
    string += f'{metric} = {value}; '
    
string

99/99 [==============================] - 3s 32ms/step - loss: 1.3740 - sparse_categorical_accuracy: 0.8885


'loss = 1.3739680051803589; sparse_categorical_accuracy = 0.8884705305099487; '